In [ ]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
from eo_tools.S1.core import S1IWSwath, align, coregister
from eo_tools.S1.util import presum, boxcar
from eo_tools.util import show_cog
import numpy as np
import logging
logging.basicConfig(level=logging.INFO)
import matplotlib.pyplot as plt
import numpy as np
import rasterio

from folium import LayerControl
# import xarray as xr

import logging

log = logging.getLogger(__name__)

In [ ]:
data_dir = "/data/S1"
primary_dir = f"{data_dir}/S1A_IW_SLC__1SDV_20230903T183344_20230903T183412_050167_0609B4_100E.SAFE"
secondary_dir = f"{data_dir}/S1A_IW_SLC__1SDV_20230915T183345_20230915T183413_050342_060F9F_85A4.SAFE"
out_dir = f"/data/res/test_new_proc"
iw = 1
pol = "vv"
up = 2

burst_min = 1
burst_max = 2

In [ ]:
# import rasterio
# with rasterio.open("/data/res/test_new_proc/primary.tif") as ds:
#     img = ds.read(1)
# with rasterio.open("/data/res/test_new_proc/secondary.tif") as ds:
#     img2 = ds.read(1)

# plt.figure(figsize=(10, 10))
# plt.imshow(np.angle(img*img2.conj())[:,::8].T, interpolation="none", vmax = 1)
# plt.colorbar(fraction=0.046, pad=0.04)

In [ ]:
from eo_tools.S1.process import preprocess_insar_iw
preprocess_insar_iw(primary_dir, secondary_dir, out_dir, burst_min, burst_max)

In [ ]:
from eo_tools.S1.process import slc2geo, interferogram, coherence
file_prm = f"{out_dir}/primary.tif"
file_sec = f"{out_dir}/secondary.tif"
file_ifg = f"{out_dir}/ifg_rdr.tif"
file_phi = f"{out_dir}/phi_geo.tif"
file_coh_r = f"{out_dir}/coh_rdr.tif"
file_coh = f"{out_dir}/coh_geo.tif"
file_lut = f"{out_dir}/lut.tif"
interferogram(file_prm, file_sec, file_ifg)
coherence(file_prm, file_sec, file_coh_r, box_size=5, magnitude=True)
slc2geo(file_ifg, file_lut, file_phi, 2, 8, 3, True)
slc2geo(file_coh_r, file_lut, file_coh, 2, 8, 3, False)

In [ ]:
import xarray as xr
coh = xr.open_dataset(file_coh, engine="rasterio")
plt.figure(figsize=(10, 10))
plt.imshow(coh["band_data"][0], interpolation="none")
plt.colorbar(fraction=0.046, pad=0.04)

In [ ]:
import xarray as xr
phi = xr.open_dataset(file_phi, engine="rasterio")
plt.figure(figsize=(10, 10))
plt.imshow(phi["band_data"][0], interpolation="none")
plt.colorbar(fraction=0.046, pad=0.04)

In [ ]:
# import folium
# m = Map()
# tile = folium.TileLayer(
#         tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
#         attr = 'Esri',
#         name = 'Esri Satellite',
#         overlay = False,
#         control = True
#        ).add_to(m)

# # hack: avoid titiler caching the tiles
# scale_phi = np.pi + np.random.rand() * 1e-6
# rnd_coh = np.random.rand() * 1e-6

# show_cog("/data/res/test_merge_coh2.tif", m,rescale=f"0,{1+rnd_coh}")
# show_cog("/data/res/test_merge_phi2.tif", m,rescale=f"-{scale_phi}, {scale_phi}",colormap=palette_phi())
# LayerControl().add_to(m)
m